In [74]:
%pip install numpy scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Código inicial

In [75]:
import random
import numpy as np

from sklearn.utils import Bunch
from scipy.sparse import csr_matrix
from sklearn.metrics import f1_score
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_20newsgroups
from typing import List, Dict, Optional, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}
idx = 4811
cossim = cosine_similarity(X_train[idx], X_train)[0]
np.sort(cossim)[::-1]
np.argsort(cossim)[::-1]

clf = MultinomialNB()
clf.fit(X_train, y_train)
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred = clf.predict(X_test)
f1_score_result = f1_score(y_test, y_pred, average='macro')

print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')
print(f"F1 Score: {f1_score_result:.4f}")

shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631
F1 Score: 0.5854


### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

#### Punto 1

In [77]:
def get_random_doc_indices(n: int, total_docs: int, seed: Optional[int] = None) -> List[int]:
    if seed is not None:
        random.seed(seed)
    return random.sample(range(total_docs), n)


def get_most_similar_docs(X: csr_matrix, idx: int, top_n: int = 5) -> List[Dict[str, Any]]:
    similarities = cosine_similarity(X[idx], X)[0]
    sorted_indices = np.argsort(similarities)[::-1]
    most_similar = []
    for i in sorted_indices[1:top_n+1]:
        most_similar.append({'doc_index': i, 'cos_sim': float(similarities[i])})
    return most_similar


def print_similar_docs_info(idx: int, similar_docs: List[Dict[str, Any]], dataset: Bunch) -> None:
    base_class = dataset.target_names[dataset.target[idx]]
    base_text = dataset.data[idx][:300].replace('\n', ' ')
    print(f"\n>>> Documento base (idx={idx}) - Clase: {base_class}")
    print(f"Contenido (primeros 300 caracteres): {base_text}\n")

    for i, doc in enumerate(similar_docs, start=1):
        doc_idx = doc['doc_index']
        cos_sim = doc['cos_sim']
        doc_class = dataset.target_names[dataset.target[doc_idx]]
        doc_text = dataset.data[doc_idx][:300].replace('\n', ' ')
        print(f">>> Similar #{i}: idx={doc_idx} - Clase: {doc_class} - Similitud coseno: {cos_sim:.4f}")
        print(f"Contenido: {doc_text}\n")


random_indices = get_random_doc_indices(n=5, total_docs=X_train.shape[0], seed=42)

for idx in random_indices:
    similar_docs = get_most_similar_docs(X_train, idx, top_n=5)
    print_similar_docs_info(idx, similar_docs, newsgroups_train)
    print("-" * 80)



>>> Documento base (idx=10476) - Clase: rec.sport.hockey
Contenido (primeros 300 caracteres): This is a general question for US readers:  How extensive is the playoff coverage down there?  In Canada, it is almost impossible not to watch a series on TV (ie the only two series I have not had an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because I'm in the wrong time zo

>>> Similar #1: idx=5064 - Clase: rec.sport.hockey - Similitud coseno: 0.2250
Contenido:  I only have one comment on this:  You call this a *classic* playoff year and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston fan and I even realize that Chicago-Detroit games are THE most exciting games to watch.

>>> Similar #2: idx=9623 - Clase: talk.politics.mideast - Similitud coseno: 0.2174
Contenido: Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012                  Prelude to Current Events in Nagorno-Karabakh          +-------------------------------------------

Los resultados muestran que la similitud coseno entre vectores TF-IDF permite recuperar documentos con contenido relacionado al documento tomado de ejemplo, tanto en términos semánticos como de categoría. En los casos en los que la similitud es relativamente alta, los documentos más cercanos suelen pertenecer a la misma clase y comparten vocabulario específico del tema, lo que valida la efectividad de la representación vectorial. 

Sin embargo, cuando la similitud es baja, los documentos recuperados frecuentemente no tienen relación semántica clara ni coinciden en la categoría, lo que indica que en esos casos el vectorizador no logra capturar adecuadamente el significado del texto. Por lo tanto, la utilidad de esta métrica está fuertemente condicionada por la calidad de la vectorización y por el umbral de similitud alcanzado.

#### Punto 2

In [78]:
def vectorize_data(train_data: list, test_data: list) -> Tuple[TfidfVectorizer, csr_matrix, csr_matrix]:
    vectorizer = TfidfVectorizer(
        stop_words='english',
        ngram_range=(1, 2),
    )
    X_train = vectorizer.fit_transform(train_data)
    X_test = vectorizer.transform(test_data)
    return vectorizer, X_train, X_test


def train_classifier(model: ClassifierMixin, X_train: csr_matrix, y_train: list) -> ClassifierMixin:
    model.fit(X_train, y_train)
    return model


def evaluate_model(model: ClassifierMixin, X_test: csr_matrix, y_test: list) -> float:
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average='macro')


vectorizer, X_train_vec, X_test_vec = vectorize_data(newsgroups_train.data, newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

multinomial_model = train_classifier(MultinomialNB(), X_train_vec, y_train)
f1_multinomial = evaluate_model(multinomial_model, X_test_vec, y_test)
print(f"F1-score macro (MultinomialNB): {f1_multinomial:.4f}")

complement_model = train_classifier(ComplementNB(), X_train_vec, y_train)
f1_complement = evaluate_model(complement_model, X_test_vec, y_test)
print(f"F1-score macro (ComplementNB): {f1_complement:.4f}")


F1-score macro (MultinomialNB): 0.6425
F1-score macro (ComplementNB): 0.7039


Luego de evaluar distintas configuraciones del vectorizador TfidfVectorizer, se concluye que la eliminación de stop words en inglés es la que más impacto tiene en el desempeño del modelo, mejorando el F1-score macro de 0.5854 a 0.6425.

Adicionalmente, la incorporación de bigramas mediante ngram_range=(1, 2) proporciona una mejora adicional, alcanzando un F1-score macro de 0.7039 con el clasificador ComplementNB.

Otras modificaciones como sublinear_tf, min_df o max_df no mejoraron el rendimiento en este caso y, en algunos casos, lo empeoraron. Por lo tanto, la mejor configuración se logra combinando eliminación de stopwords con uso de unigramas y bigramas, manteniendo el resto de los parámetros por defecto.

#### Punto 3

In [79]:
def get_idx2word_mapping(vectorizer: TfidfVectorizer) -> Dict[int, str]:
    return {v: k for k,v in vectorizer.vocabulary_.items()}


def get_most_similar_words(
        X_T: csr_matrix, vectorizer: TfidfVectorizer, word: str, top_n: int = 5
    ) -> List[Dict[str, Any]]:
    if word not in vectorizer.vocabulary_:
        raise ValueError(f"La palabra '{word}' no está en el vocabulario del vectorizador.")    
    word_idx = vectorizer.vocabulary_[word]
    similarities = cosine_similarity(X_T[word_idx], X_T)[0]
    sorted_indices = np.argsort(similarities)[::-1]
    idx2word = get_idx2word_mapping(vectorizer)
    most_similar = []
    for idx in sorted_indices[1:top_n+1]:
        similar_word = idx2word[idx]
        most_similar.append({'word': similar_word, 'cos_sim': float(similarities[idx])})    
    return most_similar


X_train_T = X_train_vec.transpose()
words_to_test = ['hockey', 'god', 'car', 'computer', 'sports',]
for word in words_to_test:
    try:
        similar_words = get_most_similar_words(X_train_T, vectorizer=vectorizer, word=word, top_n=5)
        print(f"Palabras similares a '{word}':")
        for similar_word in similar_words:
            print(f"  - {similar_word['word']} (Similitud: {similar_word['cos_sim']:.4f})")
        print("-" * 80)
    except ValueError as e:
        print(e)


Palabras similares a 'hockey':
  - hockey league (Similitud: 0.3003)
  - ncaa (Similitud: 0.2988)
  - nhl (Similitud: 0.2938)
  - hockey players (Similitud: 0.2911)
  - market hockey (Similitud: 0.2884)
--------------------------------------------------------------------------------
Palabras similares a 'god':
  - god god (Similitud: 0.3086)
  - god does (Similitud: 0.3002)
  - bible (Similitud: 0.2995)
  - jesus (Similitud: 0.2977)
  - christ (Similitud: 0.2962)
--------------------------------------------------------------------------------
Palabras similares a 'car':
  - car car (Similitud: 0.2792)
  - new car (Similitud: 0.2653)
  - bought car (Similitud: 0.2602)
  - car dealer (Similitud: 0.2212)
  - car like (Similitud: 0.2122)
--------------------------------------------------------------------------------
Palabras similares a 'computer':
  - computer science (Similitud: 0.2355)
  - computer graphics (Similitud: 0.2342)
  - turn computer (Similitud: 0.2062)
  - cuz everytime (Si

Como se vio en clase, al transponer la matriz documento-término se obtiene una matriz término-documento, donde cada fila representa el patrón de ocurrencia de una palabra a lo largo de todos los documentos del corpus. Esta representación permite aplicar la similitud coseno entre palabras, interpretándolas como vectores de contexto.

Los resultados obtenidos muestran que esta aproximación capta posibles relaciones semánticas. Por ejemplo, la palabra "god" se asocia fuertemente con "jesus", "bible" y "christ", lo que refleja que estos términos aparecen frecuentemente en documentos similares, reforzando la idea de que comparten contexto semántico. De forma análoga, "hockey" se vincula con "nhl" (National Hockey League), "hockey league" y "ncaa" (National Collegiate Athletic Association), todos términos específicos del dominio deportivo.

Esto sugiere que la vectorización TF-IDF no solo permite representar documentos, sino que también habilita el análisis de relaciones entre palabras a partir de su distribución en el corpus. Aunque esta técnica no capta significado profundo como lo haría un modelo de embeddings, es suficiente para identificar asociaciones contextuales claras dentro del dominio del dataset.